# PRAC PoPETs 2024 artifact

## Install necessary libraries

In [1]:
!pip install matplotlib
!pip install pandas
!pip install mercury
!pip install --upgrade mergetb

## Import necessary libraries

In [2]:
import time, socket
from timeout_timer import timeout, TimeoutInterrupt

from mergetb.workspace import User, Experiment, Project
from mergetb.realize import Realization
from mergetb.materialize import Materialization
from mergetb.grpc_client import MergeGRPCError
from mergetb.types import StatusType

## Log in as user

In [3]:
username='alix'
passw='SmaduMestivel13'

try:
    u = User(username, passw)
    u.login()
except MergeGRPCError as e:
    print(f"Error logging in: {e}")
else:
    print("Logged in as user", u.username)

Logged in as user alix


## Find or create experiment

In [4]:
try:
    # load the user's personal project
    p = Project(username)
    resp = p.get()

    # look for the experiment, create if it does not exist
    for exp in resp.project.experiments:
        if exp == 'prac':
            print('experiment prac already exists')
            break
    else:
        e = Experiment('prac', username, description='Replicating prac artifact')
        e.create()
        print('experiment prac created')
    
except MergeGRPCError as e:
    print(e)
#proj = m.find_project(username)
#exp = m.find_experiment('dpra', proj.name)
#if exp == None:
#    exp = m.create_experiment('dpra', proj.name, 'Replicating DPRA artifact')

experiment prac already exists


## Push model, allocate resources and attach the experiment to the XDC

In [5]:
RLZ_TIMEOUT=15
MTZ_TIMEOUT=60

try:
    exp = Experiment('prac', username)
    rev = exp.push_model('1728582833596.py').revision
    print("Pushed experiment revision", rev)
    time.sleep(5)
    
    # reserve resources through a lease named 'v1' (name is arbitrary)
    exp.reserve('v1', revision=rev)

    # wait up for 10 sec for reservation to succeed
    try:
        rlz = Realization('v1', exp.name, exp.project)
        with timeout(RLZ_TIMEOUT):
            while True:
                resp = rlz.get()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('leased resources')
                    break
    except TimeoutInterrupt:
        raise Exception('reservation did not succeed within %d seconds' % RLZ_TIMEOUT)

    # activate experiment on this lease
    exp.activate('v1')

    # wait up to 60 sec for activation to succeed
    try:
        mtz = Materialization('v1', exp.name, exp.project)
        with timeout(MTZ_TIMEOUT):
            while True:
                resp = mtz.get_status()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('started experiment')
                    break
    except TimeoutInterrupt:
        raise Exception('activation did not succeed within %d seconds' % MTZ_TIMEOUT)

    hostname = socket.gethostname().split('-')[0].split('.')[0]
    exp.attach_xdc('v1', hostname, username)
    time.sleep(15)
    print("Attached experiment is now running")
except MergeGRPCError as e:
    print(e)

Pushed experiment revision ab58bbef69be01275b373d644015150315c25d80

An Uncategorized MergeTB API Error Occurred:
    Message:     realization already exists
    Status Code: 6 (ALREADY_EXISTS)
    Details:     None


## Run commands in experiment to clone the artifact, install prerequisites, fix some bugs and execute one script in the artifact

In [6]:
installpackages="sudo apt-get install git && sudo apt-get install python3-venv && sudo apt-get install docker.io && sudo systemctl start docker && sudo systemctl enable docker && sudo apt-get install bc -y"
clonerepo="git clone https://git-crysp.uwaterloo.ca/iang/prac"

print("Installing Linux packages, this step may take a while ...")
(lines, elines) = exp.exec_on_node(username, "a", installpackages)
for line in lines:
    print(line.strip())
print("Cloning artifact repository");
(lines, elines) = exp.exec_on_node(username, "a", clonerepo)
for line in lines:
     print(line.strip())
print("Will now execute the code for fig 6, 7, 8 and Tab 3, 4; it will take approximatively 50h")
runcode="cd prac; git checkout popets-repro; cd repro; sudo ./build-all-dockers; sudo ./repro-all-dockers test; sudo ./repro-all-dockers all 1"
(lines, elines) = exp.exec_on_node(username, "a", runcode)
for line in lines:
     print(line.strip())
print('end')


Installing Linux packages, this step may take a while ...
Reading package lists...
Building dependency tree...
Reading state information...
git is already the newest version (1:2.30.2-1+deb11u2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python3-venv is already the newest version (3.9.2-3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
docker.io is already the newest version (20.10.5+dfsg1-1+deb11u2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
bc
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 109 kB of archives.
After this operation, 247 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian b

In [12]:
installpackages="sudo apt-get install git && sudo apt-get install python3-venv && curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh && . '$HOME/.cargo/env' && rustup update -y"
clonerepo="git clone https://git-crysp.uwaterloo.ca/iang/prac"

print("Installing Linux packages, this step may take a while ...")
(lines, elines) = exp.exec_on_node(username, "b", installpackages)
for line in lines:
    print(line.strip())
print("Cloning artifact repository");
(lines, elines) = exp.exec_on_node(username, "b", clonerepo)
for line in lines:
     print(line.strip())

print("Will now execute the code for node b. This code will make an error necessary for the artifact")
runcode="cd prac; git checkout popets-repro; cd repro; sudo ./setup-fig9-livenet; ./build-fig9-livenet "
(lines, elines) = exp.exec_on_node(username, "b", runcode)
for line in lines:
     print(line.strip())


print('end')

Installing Linux packages, this step may take a while ...
Cloning artifact repository
Will now execute the code for node b. This code will make an error necessary for the artifact
end


In [13]:
fixmanifest = "cd prac/repro/comps/ramen/communicator/; sed -i '6,9c\\authors = [\"alix\"]\nlicense = \"MIT\"\nreadme = \"readme.md\"\nrepository = \"\"' Cargo.toml"
print("Applying artifact fixes")
(lines, elines) = exp.exec_on_node(username, "b", fixmanifest)
for line in lines:
     print(line.strip())

print('end')


Applying artifact fixes
end


In [14]:
print("Run this after the artifact fixes")
runcode="cd prac; cd repro; sudo ./setup-fig9-livenet; ./build-fig9-livenet "
(lines, elines) = exp.exec_on_node(username, "b", runcode)
for line in lines:
     print(line.strip())

print('end')

Run this after the artifact fixes, it will take 10 min
end


In [16]:
installpackages="sudo apt-get install git && sudo apt-get install python3-venv && curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh && . '$HOME/.cargo/env' && rustup update -y"
clonerepo="git clone https://git-crysp.uwaterloo.ca/iang/prac"

print("Installing Linux packages, this step may take a while ...")
(lines, elines) = exp.exec_on_node(username, "c", installpackages)
for line in lines:
    print(line.strip())
print("Cloning artifact repository");
(lines, elines) = exp.exec_on_node(username, "c", clonerepo)
for line in lines:
     print(line.strip())

print("Will now execute the code for node b. This code will make an error necessary for the artifact")
runcode="cd prac; git checkout popets-repro; cd repro; sudo ./setup-fig9-livenet; ./build-fig9-livenet "
(lines, elines) = exp.exec_on_node(username, "c", runcode)
for line in lines:
     print(line.strip())
print('end')


Installing Linux packages, this step may take a while ...
Cloning artifact repository
Will now execute the code for node b. This code will make an error necessary for the artifact
end


In [17]:
fixmanifest = "cd prac/repro/comps/ramen/communicator/; sed -i '6,9c\\authors = [\"alix\"]\nlicense = \"MIT\"\nreadme = \"readme.md\"\nrepository = \"\"' Cargo.toml"
print("Applying artifact fixes")
(lines, elines) = exp.exec_on_node(username, "c", fixmanifest)
for line in lines:
     print(line.strip())

print('end')


Applying artifact fixes
end


In [18]:
print("Run this after the artifact fixes")
runcode="cd prac; cd repro; sudo ./setup-fig9-livenet; ./build-fig9-livenet "
(lines, elines) = exp.exec_on_node(username, "c", runcode)
for line in lines:
     print(line.strip())

print('end')

Run this after the artifact fixes, it will take 10 min
end


In [20]:
installpackages="sudo apt-get install git && sudo apt-get install python3-venv && curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh && . '$HOME/.cargo/env' && rustup update -y"
clonerepo="git clone https://git-crysp.uwaterloo.ca/iang/prac"

print("Installing Linux packages, this step may take a while ...")
(lines, elines) = exp.exec_on_node(username, "d", installpackages)
for line in lines:
    print(line.strip())
print("Cloning artifact repository");
(lines, elines) = exp.exec_on_node(username, "d", clonerepo)
for line in lines:
     print(line.strip())

print("Will now execute the code for node b. This code will make an error necessary for the artifact")
runcode="cd prac; git checkout popets-repro; cd repro; sudo ./setup-fig9-livenet; ./build-fig9-livenet "
(lines, elines) = exp.exec_on_node(username, "d", runcode)
for line in lines:
     print(line.strip())
print('end')


Installing Linux packages, this step may take a while ...
Cloning artifact repository
Will now execute the code for node b. This code will make an error necessary for the artifact
end


In [21]:
fixmanifest = "cd prac/repro/comps/ramen/communicator/; sed -i '6,9c\\authors = [\"alix\"]\nlicense = \"MIT\"\nreadme = \"readme.md\"\nrepository = \"\"' Cargo.toml"
print("Applying artifact fixes")
(lines, elines) = exp.exec_on_node(username, "d", fixmanifest)
for line in lines:
     print(line.strip())
print('end')



Applying artifact fixes
end


In [22]:
print("Run this after the artifact fixes")
runcode="cd prac; cd repro; sudo ./setup-fig9-livenet; ./build-fig9-livenet "
(lines, elines) = exp.exec_on_node(username, "d", runcode)
for line in lines:
     print(line.strip())

print('end')

Run this after the artifact fixes, it will take 10 min
end


In [23]:
print("We will run on the main node again, it will take 10 min")
runcode="cd prac; git checkout popets-repro; cd repro; ./clone-fig9-livenet; export PRAC_SSH_P0_USERHOST=alix@b; export PRAC_SSH_P0_SSHOPTS=; export PRAC_SSH_P0_IP= b; export PRAC_SSH_P0_DIR=prac; export PRAC_SSH_P1_USERHOST=alix@c; export PRAC_SSH_P1_SSHOPTS=; export PRAC_SSH_P1_IP= c; export PRAC_SSH_P1_DIR=prac; export PRAC_SSH_P2_USERHOST=alix@d; export PRAC_SSH_P2_SSHOPTS=; export PRAC_SSH_P2_IP= d; export PRAC_SSH_P2_DIR=prac; sudo ./repro-fig9-livenet 1 "
(lines, elines) = exp.exec_on_node(username, "a", runcode)
for line in lines:
     print(line.strip())
print('end')

We will run on the main node again, it will take 10 min
Your branch is up to date with 'origin/popets-repro'.
Running PRAC experiments...

Tue 15 Oct 2024 07:45:26 PM UTC: Starting experiments
Running Figure 9 experiments...
Tue 15 Oct 2024 07:45:26 PM UTC: Running -p m:1033 h:7497 a:3 s:118 r1:1 r2:6 r3:57 i1:6 i2:57 i1.3:2 i2.3:4 i3.3:57 c:610 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running heapsampler 64 10 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running -p m:2121 h:15561 a:6 s:246 r1:1 r2:6 r3:121 i1:6 i2:121 i1.3:2 i2.3:4 i3.3:121 c:1250 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running heapsampler 128 10 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running -p m:4297 h:31689 a:12 s:502 r1:1 r2:6 r3:249 i1:6 i2:249 i1.3:2 i2.3:4 i3.3:249 c:2530 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running heapsampler 256 10 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running -p m:8649 h:63945 a:24 s:1014 r1:1 r2:6 r3:505 i1:6 i2:505 i1.3:2 i2.3:4 i3.3:505 c:5090 ...
Tue 15 Oct 2024 07:45:26 PM UTC: Running heapsampler 512 1

## Display results

In [ ]:

print("Run this after the artifact fixes")
cd /proj/alix/…; cat *.out

## Return resources and if you so wish delete the experiment

In [ ]:
#try:
#    exp.relinquish('v1')
#except MergeGRPCError as e:
#    print(e)
#else:
#    print("experiment relinquished")
#exp.delete()